In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# iWildCam2020 revised1

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install tridentx --upgrade
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet

由於這個競賽原始數據非常巨大，也因此我們無法直接載入，我已經將所需要的部分檔案上傳到我的google drive並開放權限。然後您只需要利用trident api中的download_file_from_google_drive函數即可方便的下載，預設下載目錄會是trident api的主資料夾下方的downloads資料夾。下載後，我們可以透過讀讀取標註json檔來解讀標註內容。

了解你的數據是在做數據分析前非常重要的階段，我們也稱這個階段為DAE (Data Exploration Analysis)，如果是處理表格型態的內容，最方便的莫過於pandas。身為數據科學家，千萬不要講到熊貓直覺反應是要點外賣。pandas最大的好處在於它整合了多種數據來源的讀取與寫入，即使是像這次屬於非結構數據的json檔，它也能轉換成表格型態數據(DataFrame)

In [ ]:
import pandas as pd

with open('../input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_data = json.load(json_file)


with open('../input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as test_json_file:
    test_data = json.load(test_json_file)

    
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                         'category_id': [item['category_id'] for item in train_data['annotations']],
                         'image_id': [item['image_id'] for item in train_data['annotations']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})
df_test = pd.DataFrame({'image_id': [item['id'] for item in train_data['images']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})



df_train

接下來可以透過類似的方式將train_data['categories']取出建構成新的dtaframe，並列印一下，這才知道，原來出現最多次的是「人類」這種動物。而除了人類這種動物之外，是一種叫做「眼斑吐綬雞(meleagris ocellata)」的動物。而更變態的是，標註中的類別還出現了0次的動物，而且數量還不少。所以我們這時可以做個過濾，排除掉不存在於圖片標註中的類別

我們可以利用is_in這個函數來進行欄位的篩選，這才知道像是舊世界綠猴(chlorocebus pygerythrus)等動物其實只出現過一次，而原本類別檔中出現最多次的人類竟然不見了，可見category中的數量是完全不可信的，只能參考它的category_id以及名稱之間的對應。更糟的是，雖然訓練及與測試集的類別表看起來是一致的，但是由於它不可能給測試集的標註，我們只能根據category中數量大於零來作為判斷，然後一去重複比對之下，竟然出現測試集出現了意料之外的動物的這種劇情。我檢查半天我沒寫錯，查了一下討論區，看到了以下留言：

https://www.kaggle.com/c/iwildcam-2020-fgvc7/discussion/143071

好吧，只能暫時相信出題方了。而也因此我開始擔心圖片annotations中的註記是否跟圖片一致....

In [ ]:
df_category_train=pd.DataFrame({'id': [item['id'] for item in train_data['categories']],
                         'name': [item['name'] for item in train_data['categories']],
                         'count': [item['count'] for item in train_data['categories']]})

df_category_test=pd.DataFrame({'id': [item['id'] for item in test_data['categories']],
                         'name': [item['name'] for item in test_data['categories']],
                         'count': [item['count'] for item in test_data['categories']]})

df_category_train=df_category_train.sort_values(['count'],ascending=False) 
print(df_category_train)
df_category_test=df_category_test.sort_values(['count'],ascending=False) 
print(df_category_test)

#基於標註檔，產生有在標註檔內的所有圖片的清單，進行去重複(set)、排序(sorted)以及轉換成清單(list)
animal_category_lists=list(sorted(set([item['category_id'] for item in train_data['annotations']])))


#進行篩選
df_category_train=df_category_train[df_category_train['id'].isin(animal_category_lists)]
print(df_category_train)

df_category_test=df_category_test[df_category_test['count']>0]
print(df_category_test)

animal_category_lists_train=[category_id.item() for category_id in df_category_train[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_train[:5])

animal_category_lists_test=[category_id.item() for category_id in df_category_test[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_test[:5])

#檢查是不是所有df_category_test數量不為零的動物都有出現在df_category_train的類別代號中
category_missing_list=[category_id for category_id in animal_category_lists_test if category_id not in animal_category_lists_train]
print(category_missing_list)





除了數量外，我們也要注意到，出現最多的動物以及出現最少的動物，其圖片數量相差了1.6萬倍，我們可以透過value_counts()來計算次數分配表，然後透過pandas中的plot自動畫圖。

由於有許多動物只出現一次，為了避免包含只有出現過一次的動物的圖片因為數據增強或是其他因素意外的沒有呈現出該動物特徵而讓整個動物類別都無法識別，因此，我打算先把所有出現頻率超低的動物先列出來，然後將包含這些動物的圖片多複製幾分與原圖片集融合，這樣就比較不會有前述的問題。

In [ ]:
rare_animals=OrderedDict()
df_animal_frequency=df_train['category_id'].value_counts()
df_animal_frequency=df_animal_frequency[df_animal_frequency<=10]
for item in  df_animal_frequency.iteritems() :
    print(item)
    rare_animals[item[0]]=item[1]

我們可以透過以下語法抓出所有圖片，以及統計一下數量。經過去重複，還好去重前後數量都是217959，這個數字是合的。沒有出現我覺得最討人厭的一張照片出現兩種動物，也因此，這個題目是可以符合圖像識別的要求(但這只是最低要求，我只是要用它做個baseline以及展示一下多類別不均衡如何解)，那我們就開始來做出baseline吧。

In [ ]:
import glob
#透過glob所全部train資料夾中所有可用圖片
imgs=glob.glob('../input/iwildcam-2020-fgvc7/train/*.jpg')
print(len(imgs))
print(imgs[:5])

#將圖檔路徑去除資料夾部分後進行去重複
img_pathes=[img.split('/')[-1] for img in imgs]
img_pathes=list(sorted(set(img_pathes)))
print(len(img_pathes))
print(img_pathes[:5])



但是仔細看可以發現，我們的動物類別id並不是連續數字，中間是有跳過的，而且既然出題方都說了測試集不會出現意料之外的動物，那麼我們就按照訓練集出來的類別清單(animal_category_lists_train)來建構標籤囉。

In [ ]:
label2category=OrderedDict()
category2label=OrderedDict()
#產生能將category_id轉label的字典
for i in range(len(animal_category_lists_train)):
    category2label[animal_category_lists_train[i]]=i
    label2category[i]=animal_category_lists_train[i]

#建構出轉成標籤id
label_idxes=[category2label[item['category_id']] for item in train_data['annotations']]
image_pathes=['../input/iwildcam-2020-fgvc7/train/'+item['file_name'] for item in train_data['images']]

print('label_idxes',label_idxes[:5])
print('image_pathes',image_pathes[:5])
print('label2category',list(label2category.items())[:5])
print('category2label',list(category2label.items())[:5])

In [ ]:

rare_images=[]
rare_labels=[]
for i in range(len(image_pathes)):
    img=image_pathes[i]
    label=label_idxes[i]
    if label2category[label] in rare_animals:
        cnt=rare_animals[label2category[label]]
        for n in range(int(20.0/cnt)):
            rare_images.append(img)
            rare_labels.append(label) 
        
print(len(rare_images))
print(len(rare_labels))
print(rare_images[:5])
print(rare_labels[:5])

In [ ]:

img_ds=ImageDataset(image_pathes+5*rare_images,symbol='image')
label_ds=LabelDataset(label_idxes+5*rare_labels,symbol='label')

#與Iterator構成data provider
data_provider=DataProvider(traindata=Iterator(data=img_ds,label=label_ds))

data_provider.image_transform_funcs=[
    Resize((224,224)),
    CLAHE(),
    RandomAdjustGamma(scale=(0.8,1.2)),#調整明暗
    RandomAdjustHue(scale=(-0.2,0.2)),#調整色相
    RandomAdjustSaturation(scale=(0.8,1.2)),#調整飽和度
    SaltPepperNoise(0.005, keep_prob=0.5),#加入胡椒鹽噪音
    RandomErasing(size_range=(0.05, 0.2), transparency_range=(0.4, 0.8), transparancy_ratio=1.0, keep_prob=0.5), #加入隨機擦去
    RandomTransformAffine(rotation_range=45, zoom_range=0.00, shift_range=0.00, shear_range=0.2, random_flip=0.15),#隨機仿射變換
    Normalize(127.5,127.5)] #標準化


data,labels=data_provider.next()
print(data.shape)
print(labels)



用preview_images預覽一下圖片，一看嚇到，是我運氣不好嗎?一預覽就看到一堆純黑的圖。

In [ ]:
%%time
data_provider.preview_images()

In [ ]:

from trident.models import efficientnet

net1=efficientnet.EfficientNetB0(pretrained=True,include_top=False,classes=len(animal_category_lists_train),input_shape=(3,224,224),freeze_features=True)
net1.model.add_module('last_conv',Conv2d_Block((3,3),num_filters=len(animal_category_lists_train),use_bias=False,activation=None, normalization='l2'))
cam=ShortCut(
    Identity(),
    Sequential(
    GlobalAvgPool2d(),
    Reshape((len(animal_category_lists_train),1,1)),
    Conv2d((1,1),num_filters=len(animal_category_lists_train),use_bias=False,activation=None)
    )
,mode='dot'
)

net1.model.add_module('cam',cam)
net1.model.add_module('aggregate1',Aggregation('sum',axis=2))
net1.model.add_module('aggregate2',Aggregation('sum',axis=3))
net1.model.add_module('reshape',Reshape((len(animal_category_lists_train))))
net1.model.add_module('sigmoid',Sigmoid())
net1.model.add_module('fc',Dense((len(animal_category_lists_train))))
net1.model.add_module('softmax',SoftMax(axis=-1,add_noise=True,noise_intensity=0.12))
net1.summary()

net1.model.block7a.trainable=True

In [ ]:
from trident.models import visual_transformer
vit=visual_transformer.VisionTransformer_small(input_shape=(3,224,224),patch_size=16,num_classes=len(animal_category_lists_train))
vit.model.trainable=True
vit.summary()

In [ ]:
is_resume=False  #是否接續訓練

#如果是接續訓練且存在歷史存檔，則載入之
if is_resume:
    if os.path.exists('./Models/revised2_net1.pth'):
        net1.load_model('./Models/revised2_net1.pth')
        net1.model.block7a.trainable=True
        print('net1 loaded!!')

    if os.path.exists('./Models/revised2_vit.pth'):
        vit.load_model('./Models/revised2_vit.pth')
        print('vit loaded!!')

這兩個baseline主要是用來確認，哪一個類別不平衡的校正策略比較好。請注意，由於建模圖片數量又多，圖檔尺寸又超大，即使只跑一個epoch都要很久，為了不浪費gpu時數資源，請注意，建議要把Resize類的轉換放在第一個，時間會差非常非常多...還有不要用原來的start_now()，改用only_steps(num_steps=步數)只需要指定步數即可。

In [ ]:
net1.load_model('../input/iwildcam2020-revised2/Models/revised2_net1.pth.tar')
net1.with_optimizer(optimizer=AdaBelief,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True))\
.with_loss(FocalLoss,loss_weight=0.5)\
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy',print_only=True)\
.with_regularizer('l2',reg_weight=5e-6)\
.adjust_learning_rate_scheduling(200,unit='batch',new_value=5e-4)\
.with_accumulate_grads(5)\
.with_callbacks(MixupCallback(alpha= 1,loss_criterion=CrossEntropyLoss,loss_weight=0.5))\
.with_learning_rate_scheduler(StepLR(frequency=1000,unit='batch',gamma=0.5))\
.with_model_save_path('./Models/revised2_net1.pth')\
.with_automatic_mixed_precision_training()


#transformer一樣也是可以使用mixup的
vit.load_model('../input/iwildcam2020-revised2/Models/revised2_vit.pth.tar')
vit.with_optimizer(optimizer=DiffGrad,lr=2e-4,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True))\
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy',print_only=True)\
.with_regularizer('l2',reg_weight=5e-6)\
.with_accumulate_grads(5)\
.with_grad_clipping(3)\
.with_callbacks(MixupCallback(alpha= 1,loss_criterion=CrossEntropyLoss,loss_weight=0.5))\
.with_learning_rate_scheduler(StepLR(frequency=1000,unit='batch',gamma=0.5))\
.with_model_save_path('./Models/revised2_vit.pth')\
.with_automatic_mixed_precision_training()


#如果兩個模型都做mixup，批次設定到64，gpu會爆掉
plan=TrainingPlan()\
    .add_training_item(net1,name='net1')\
    .add_training_item(vit,name='vit')\
    .with_data_loader(data_provider)\
    .with_batch_size(32)\
    .repeat_epochs(2)\
    .print_gradients_scheduling(20,unit='batch') \
    .print_progress_scheduling(5,unit='batch') \
    .display_loss_metric_curve_scheduling(100)\
    .save_model_scheduling(10,unit='batch')


plan.only_steps(num_steps=4000, collect_data_inteval=5)


之前的提交範例是一筆一筆的推論，結合infer_single_image，所有有人問到，難道不可以一整個批次推論嗎?能否將數據包裝成data provider來進行批次推論呢？可以的，請參考以下範例。

In [ ]:
bad_images=['86994b3e-21bc-11ea-a13a-137349068a90.jpg',
'882a533a-21bc-11ea-a13a-137349068a90.jpg',
'88a28616-21bc-11ea-a13a-137349068a90.jpg',
'88b99aae-21bc-11ea-a13a-137349068a90.jpg',
'89362ed4-21bc-11ea-a13a-137349068a90.jpg',
'8985bb98-21bc-11ea-a13a-137349068a90.jpg',
'89e09b26-21bc-11ea-a13a-137349068a90.jpg',
'8a804608-21bc-11ea-a13a-137349068a90.jpg',
'8b8e02a6-21bc-11ea-a13a-137349068a90.jpg',
'8b91394e-21bc-11ea-a13a-137349068a90.jpg',
'8cc46b6a-21bc-11ea-a13a-137349068a90.jpg',
'8d705d8a-21bc-11ea-a13a-137349068a90.jpg',
'8e930668-21bc-11ea-a13a-137349068a90.jpg',
'8e940310-21bc-11ea-a13a-137349068a90.jpg',
'8ea6a768-21bc-11ea-a13a-137349068a90.jpg',
'8fff9dc2-21bc-11ea-a13a-137349068a90.jpg',
'9044a3b8-21bc-11ea-a13a-137349068a90.jpg',
'920ee4c4-21bc-11ea-a13a-137349068a90.jpg',
'950ed288-21bc-11ea-a13a-137349068a90.jpg',
'9522d4fe-21bc-11ea-a13a-137349068a90.jpg',
'96bacf06-21bc-11ea-a13a-137349068a90.jpg',
'98552f5a-21bc-11ea-a13a-137349068a90.jpg',
'98da656c-21bc-11ea-a13a-137349068a90.jpg',
'9955d012-21bc-11ea-a13a-137349068a90.jpg']



test_imgs=glob.glob('../input/iwildcam-2020-fgvc7/test/*.jpg')
print(len(test_imgs))
test_imgs=[img_path for img_path in test_imgs if img_path.split('/')[-1] not in bad_images]
print(len(test_imgs))


img_ds=ImageDataset(test_imgs,symbol='image')
#請注意，要設定object_type=ObjectType.image_path，這樣就可以確保輸出為stype=np.string_的numpy array
imgpath_ds=ImageDataset(test_imgs,object_type=ObjectType.image_path,symbol='img_path')

#與Iterator構成data provider
#設定is_shuffle=False表示不隨機打亂
test_data_provider=DataProvider(traindata=Iterator(data=img_ds,label=imgpath_ds,is_shuffle=False))

#保留有意義的數據清洗
test_data_provider.image_transform_funcs=[
    Resize((224,224)),
    CLAHE(),
    Normalize(127.5,127.5)] #標準化


data,labels=test_data_provider.next()
print(data.shape)
print(labels)
print(labels[0].decode())
test_data_provider.preview_images()

In [ ]:
f=open('../input/iwildcam-2020-fgvc7/sample_submission.csv','r',encoding='utf-8-sig')
rows=f.readlines()
print(rows[:5])
print(rows[-5:])

submission_dict=OrderedDict()
for row in rows[1:]:
    cols=row.strip().split(',')
    if  cols[0]+'.jpg' not in bad_images:
        submission_dict[cols[0]]=None
print(len(submission_dict))

這段語法可以用來清除gpu快取，可以釋放出更多的GPU

In [ ]:
import torch
torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

In [ ]:
net1.eval()
vit.eval()

test_data_provider.batch_size=64
test_data_provider.traindata.batch_sampler.reset()

submission_rows=['Id,Category\n']
for i,(img,img_path) in tqdm(enumerate(test_data_provider)):
    out1=net1(img)

    out2=vit(img)
    #基於輸出平均值
    out3=np.argmax((out1+out2)/2,axis=1)
    #基於兩個模型評估的最大機率
    #out3=np.argmax(np.where(out1>out2,out1,out2),axis=1)
    for k in range(len(out3)):
        _,image_id,_=split_path(img_path[k].decode())
        if i==0 and k==0:
            print(image_id)
        category_id=label2category[out3[k]]
        if image_id in submission_dict and image_id+'.jpg' not in bad_images:
            submission_dict[image_id]=category_id
        #submission_rows.append('{0},{1}\n'.format(image_id,category_id))
        #if  (len(submission_rows)-1)%1000==0:
        #    print('{0}'.format(len(submission_rows)-1))
        if None not in submission_dict.value_list:
            break
  
 

        


In [ ]:
for k,v in submission_dict.items():
    submission_rows.append('{0},{1}\n'.format(k,v))
print(len(submission_rows))
with open('results/submission.csv','w',encoding='utf-8-sig') as f:
    f.writelines(submission_rows)

fr=open('results/submission.csv','r',encoding='utf-8-sig')
rows=fr.readlines()
print(rows[:3])
        